Import libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F

Reading data FASHION-MNIST and splitting into train- and test- datasets:

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive') #монтируем диск
TRAIN_PATH = '/content/gdrive/My Drive/fashion-mnist_train.csv'
TEST_PATH = '/content/gdrive/My Drive/fashion-mnist_test.csv'
# TRAIN_PATH = './fashion-mnist_train.csv'
# TEST_PATH = './fashion-mnist_test.csv'
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
X_train = train_df.values[:, 1:]
y_train = train_df.values[:, 0]

X_test = test_df.values  # [:, 1:]  # удаляем столбец 'label'

X_train_tensor = torch.FloatTensor(X_train).view(-1, 1, 28, 28).cuda()
y_train_tensor = torch.LongTensor(y_train.astype(np.int64)).cuda()

Putting our variables for NN:

In [0]:
D_in, D_out = 784, 10
BATCH_SIZE = 64
NUM_EPOCHS = 100
learning_rate = 1e-3
num_classes = 10  # количество классов, в нашем случае 10 типов одежды

NN:

In [0]:
class CNN(nn.Module):
    def __init__(self,num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.AvgPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AvgPool2d(2))
        self.fc1 = nn.Linear(7*7*32, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

Let's convert into cuda to train our NN on GPU:

In [0]:
model = CNN(num_classes).cuda()

Function to generate batches:

In [0]:
def generate_batches(X, y, batch_size=64):
    for i in range(0, X.shape[0], batch_size):
        X_batch, y_batch = X[i:i+batch_size], y[i:i+batch_size]
        yield X_batch, y_batch

Choosing optimizer SGD and loss function CrossEntropyLoss and training our NN:

In [26]:
loss_fn = torch.nn.CrossEntropyLoss(size_average=False) # loss_fn - функция потерь. Выбрали КроссЭнтропию
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate) # Метод оптимизации выбрали SGD - стохастический градиент

for epoch_num  in range(NUM_EPOCHS): # Одна эпоха (epoch) алгоритма оптимизации -- это проход по всей выборке.
    iter_num = 0                     # Одна итерация (iteration) алгоритма оптимизации -- это проход по одному батчу.
    running_loss = 0.0
    for X_batch, y_batch in generate_batches(X_train_tensor, y_train_tensor, BATCH_SIZE): # Цикл по батчам
        # forward (подсчёт ответа с текущими весами)
        y_pred = model(X_batch)

        # вычисляем loss'ы
        loss = loss_fn(y_pred, y_batch)
        
        running_loss += loss.item()
        
        # выводем качество каждые 2000 батчей 
            
        if iter_num % 100 == 99:
            print('[{}, {}] current loss: {}'.format(epoch_num, iter_num + 1, running_loss / 2000))
            running_loss = 0.0
            
        # зануляем градиенты
        optimizer.zero_grad()

        # backward (подсчёт новых градиентов)
        loss.backward()

        # обновляем веса
        optimizer.step()
        
        iter_num += 1

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


[0, 100] current loss: 4.3434234085083006
[0, 200] current loss: 2.147623987197876
[0, 300] current loss: 1.8783226747512818
[0, 400] current loss: 1.6927343711853027
[0, 500] current loss: 1.6238228273391724
[0, 600] current loss: 1.485894588947296
[0, 700] current loss: 1.4176460013389587
[0, 800] current loss: 1.3237768759727477
[0, 900] current loss: 1.336835762500763
[1, 100] current loss: 1.224245922088623
[1, 200] current loss: 1.1957165565490722
[1, 300] current loss: 1.1790027589797973
[1, 400] current loss: 1.1245672717094422
[1, 500] current loss: 1.1498492636680604
[1, 600] current loss: 1.100995904445648
[1, 700] current loss: 1.0695583786964415
[1, 800] current loss: 1.0468693509101867
[1, 900] current loss: 1.0560399651527406
[2, 100] current loss: 0.982388279914856
[2, 200] current loss: 0.9990334014892578
[2, 300] current loss: 0.9948314790725707
[2, 400] current loss: 0.9445443913936615
[2, 500] current loss: 0.9790902953147889
[2, 600] current loss: 0.941144466876983

Let's see the accuracy of each class:

In [27]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
           'Sandal', 'Shirt', 'Sneaker','Bag', 'Ankle boot']

with torch.no_grad():
    for X_batch, y_batch in generate_batches(X_train_tensor, y_train_tensor, BATCH_SIZE):
        y_pred = model(X_batch)
        _, predicted = torch.max(y_pred, 1)
        c = (predicted == y_batch).squeeze()
        for i in range(len(y_pred)):
            label = y_batch[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

class_correct_total = 0
class_correct_total_total = 0
for i in range(num_classes):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
    class_correct_total += class_correct[i]
    class_correct_total_total += class_total[i]
result = class_correct_total / class_correct_total_total
print(result)

Accuracy of T-shirt/top : 100 %
Accuracy of Trouser : 100 %
Accuracy of Pullover : 100 %
Accuracy of Dress : 100 %
Accuracy of  Coat : 100 %
Accuracy of Sandal : 100 %
Accuracy of Shirt : 100 %
Accuracy of Sneaker : 100 %
Accuracy of   Bag : 100 %
Accuracy of Ankle boot : 100 %
1.0


Prediction:

In [28]:
y_test_pred = model(torch.cuda.FloatTensor(X_test).view(-1, 1, 28, 28))
_, predicted = torch.max(y_test_pred, 1)

predicted.cuda()

tensor([0, 1, 2,  ..., 8, 8, 1], device='cuda:0')

Generating csv file with prediction answers to submit:

In [0]:
answer_df = pd.DataFrame(data=predicted.cpu().numpy(), columns=['Category'])
# answer_df.head()
answer_df['Id'] = answer_df.index

In [0]:
answer_df.to_csv('./baseline3.csv', index=False)
from google.colab import files
files.download('baseline3.csv')